In [1]:
import spotipy
import spotipy.oauth2 as oauth2
from spotipy.oauth2 import SpotifyOAuth
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import time


from dotenv import load_dotenv

import os
import re

In [2]:
load_dotenv(r'./.env')

user = os.getenv("USER_ID") 
key = os.getenv("TOKEN") 

auth_manager = SpotifyClientCredentials(client_id=user, client_secret=key)
sp = spotipy.Spotify(client_credentials_manager = auth_manager)

In [3]:
def getTrackIDs(user, playlist_id):
    track_ids = []
    playlist = sp.user_playlist(user,playlist_id)
    for item in playlist['tracks']['items']:
        track = item['track']
        track_ids.append(track['id'])
    return track_ids

track_ids = getTrackIDs('spotify', '6UJtsUVYLehiB7UxMSj2sA')

print(len(track_ids))
print(track_ids)

100
['4F1fVi7f1h1fN4FrwoyWMa', '7edvQQI4OxF0qbqWNmoHJn', '2f8JQJctq8LZtaq6Hgdgb8', '205HNJ73cgpC0LAOnuQiWT', '0a1Y99FcuV7kOzEQZg8J4y', '3q1FxEzXmsNmJrxaoICXzW', '7EO5oDxrq2dvYDgBjFvFYY', '5vJ4dXXqSskiJZMpcszYJX', '4cW0FXhN81yZ6YHVmFnsyG', '5WAqZjhyquaFf8zBZ1Stfd', '1MVFkoITWetg97ZJCrJV1I', '10kK47RnybBiGAB4XtUkmD', '7DOqkqVZuvPhKIAtVXjZo1', '0s5VzFR8CU6qoUNMjiYDh3', '4wNyrbY6B2Okii8VeKo2B2', '62O1EwuN0Sd9dCkPj6u7nm', '4U4k974pFMtIXwRMu3n4R0', '10IQPICqgiB3TeUV2UcYqz', '1X5U5wtor2IFwM1RaeDGgx', '0Jh6iWjna1wk5le0Ahn7No', '7qyTPWGS7llsgSbwXDDW7e', '6PrB4KpvuoYWtwjPC5a44S', '4f0ytSBF6dlKOIFQsfeemV', '6QfPyKRfSQBb3ccajHioEm', '6RLVnSWFkT7ywpXv0XyRhY', '23NVGbH0hoUOQJ0rtH5MOv', '66TAAKeoOD0cFpVRug9IXZ', '1vVZLmZq8gN8pPCEBLjPvD', '73Qd0RxSaQT9lR5yqyx2GD', '0sOeocOCv6IFQJwGhgAxyq', '6HTJgY7iWWlVLSDHsfHePf', '43vjYTKZg8R5LsuITJ4wVu', '03o8WSqd2K5rkGvn9IsLy2', '1WkliDsYrTr7UdNcQVDy2Q', '0rogG0xr0c3Fdxdm0co3aK', '21SKXioSYekcFCgZGV2Rd1', '3re2UyhFJBSG1L4cofzyYF', '6sR9M21Bl17LNMs07QSjyb', '3D29kj

In [4]:
def getTrackFeatures(track_id):
    track_info = sp.track(track_id)
    feature_info = sp.audio_features(track_id)

    print(track_info.keys())
    # Track info
    song_id = track_info['id']
    name = track_info['name']
    album = track_info['album']['name']
    artist = track_info['album']['artists'][0]['name']
    release_date = track_info['album']['release_date']
    duration = track_info['duration_ms']
    popularity = track_info['popularity']

    # Track features
    acousticness = feature_info[0]['acousticness']
    danceability = feature_info[0]['danceability']
    energy = feature_info[0]['energy']
    instrumentalness = feature_info[0]['instrumentalness']
    liveness = feature_info[0]['liveness']
    loudness = feature_info[0]['loudness']
    speechiness = feature_info[0]['speechiness']
    tempo = feature_info[0]['tempo']
    time_signature = feature_info[0]['time_signature']

    track_data = [
        song_id,
        name,
        album,
        artist,
        release_date,
        duration,
        popularity,
        acousticness,
        danceability,
        energy,
        instrumentalness,
        liveness,
        loudness,
        speechiness,
        tempo,
        time_signature,
    ]

    return track_data

In [12]:
getTrackFeatures(track_ids[0])

dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track_number', 'type', 'uri'])


['4F1fVi7f1h1fN4FrwoyWMa',
 'La faille',
 'La faille',
 'Jul',
 '2023-05-12',
 291280,
 72,
 0.227,
 0.941,
 0.775,
 0,
 0.0964,
 -5.151,
 0.167,
 130.028,
 4]

In [5]:
COLUMNS =  ['name','album','artist','release_date','duration','popularity','acousticness','danceability','energy',
            'instrumentalness','liveness','loudness', 'speechiness', 'tempo', 'time_signature']
track_list = []
for i, id in enumerate(track_ids):
    time.sleep(.3)
    track_data = getTrackFeatures(id)
    track_list.append(track_data)

rap_playlist = pd.DataFrame(track_list, columns=COLUMNS)
rap_playlist.head(10)

dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track_number', 'type', 'uri'])
dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track_number', 'type', 'uri'])
dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track_number', 'type', 'uri'])
dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track_number', 'type', 'uri'])
dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'explicit', 'external_ids', 'e

ValueError: 15 columns passed, passed data had 16 columns

In [19]:
rap_playlist.to_csv('data.csv')